In [2]:
from cng.utils import *
from cng.h3 import *

duckdb_install_h3()

import ibis
from ibis import _
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])

import streamlit as st
set_secrets(con, st.secrets["MINIO_KEY"], st.secrets["MINIO_SECRET"])



In [ ]:

parquet = "https://minio.carlboettiger.info/public-biodiversity/pad-us-4/pad-us-4.parquet"
con.raw_sql(f"CREATE  OR REPLACE VIEW pad4 AS SELECT Unit_Nm, row_n, geom FROM '{parquet}'")

zoom = 10

con.sql(f'''
WITH t1 AS (
  SELECT Unit_Nm, row_n, ST_Dump(geom) AS geom 
  FROM pad4
) 
SELECT Unit_Nm, row_n,
       h3_polygon_wkt_to_cells_string(UNNEST(geom).geom, {zoom}) AS h{zoom}
FROM t1
''').to_parquet(f"s3://public-biodiversity/pad-us-4/pad-h3-z{zoom}.parquet")
#parquet = "https://minio.carlboettiger.info/public-data/social-vulnerability/2022/SVI2022_US_tract.parquet" # faster with local
#con.raw_sql(f"CREATE  OR REPLACE VIEW svi AS SELECT FIPS, Shape AS geom FROM '{parquet}'")

In [3]:

(con
       .read_parquet("s3://public-biodiversity/pad-us-4/pad-h3-z10.parquet")
       .mutate(h8 = h3_cell_to_parent(_.h10.unnest(), 8))
).head() .execute()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Unit_Nm,row_n,h10,h8
0,Laguna Cartagena National Wildlife Refuge,0,"[8a4cc0b42817fff, 8a4cc0b51837fff, 8a4cc0b4201...",884cc0b429fffff
1,Laguna Cartagena National Wildlife Refuge,0,"[8a4cc0b42817fff, 8a4cc0b51837fff, 8a4cc0b4201...",884cc0b519fffff
2,Laguna Cartagena National Wildlife Refuge,0,"[8a4cc0b42817fff, 8a4cc0b51837fff, 8a4cc0b4201...",884cc0b421fffff
3,Laguna Cartagena National Wildlife Refuge,0,"[8a4cc0b42817fff, 8a4cc0b51837fff, 8a4cc0b4201...",884cc0b553fffff
4,Laguna Cartagena National Wildlife Refuge,0,"[8a4cc0b42817fff, 8a4cc0b51837fff, 8a4cc0b4201...",884cc0b429fffff


In [4]:


(con
   .read_parquet("s3://public-biodiversity/pad-us-4/pad-h3-z10.parquet")
   .mutate(h8 = h3_cell_to_parent(_.h10.unnest(), 8))
   .drop("h10")
   .distinct()
  ).to_parquet("s3://public-biodiversity/pad-us-4/pad-h3-z8.parquet")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
(con
       .read_parquet("s3://public-biodiversity/pad-us-4/pad-h3-z8.parquet")
       .mutate(h6 = h3_cell_to_parent(_.h8, 6))
       .drop("h8")
       .distinct()
      ).to_parquet("s3://public-biodiversity/pad-us-4/pad-h3-z6.parquet")

